In [1]:
from openpyxl import load_workbook
import xlwings as xw
import os 
import pandas as pd
import numpy as np
import re
import itertools
import datetime
from datetime import date,timedelta
from xlwings.constants import DeleteShiftDirection

In [3]:
# Return the day of the week as an integer, where Monday is 0 and Sunday is 6. For example, date(2002, 12, 4).weekday() == 2, a Wednesday. See also isoweekday().
if date.today().weekday() == 0:
    nowDate = date.today() - timedelta(days = 3)
else:
    nowDate = date.today() - timedelta(days = 1)
dateNum = re.sub('-', '', str(nowDate))
nowYm = str(nowDate)[:7]
nowMD = nowDate.strftime("%m%d")
print(nowDate, dateNum , nowYm , nowMD)
path = 'C:/Users/0305/Desktop/Daily PNL/'+nowYm+'/'+ nowMD 
os.chdir(path)

2021-01-25 20210125 2021-01 0125


In [6]:
# 今日日期
today = date.today()
d = timedelta(days = 1)
today = today-d
todaynum = str(today)[0:4] + str(today)[5:7] + str(today)[8:10]
print(todaynum)
todaymd = todaynum[4:]

path = 'C:/Users/0305/Desktop/Daily PNL/2021-01/'+ todaymd 
os.chdir(path)

20210125


In [7]:
# 1.修正PL balance + Closed balance + Deposit = balance
# equity change
equity_change = pd.read_csv('equity_change_vfx2.csv')
equity_change.head(20)

,Login,row,nday,BALANCE_PREV,PROFIT_CLOSED,DEPOSIT,BALANCE,PROFIT,CREDIT,EQUITY,MARGIN,MARGIN_FREE
0,6214790,26470,1,7284864,23111,0,7307975,51049,0,7359024,40687.184361,7.318337e+06
1,6214790,26169,2,7259016,25848,0,7284864,48259,0,7333123,24620.214674,7.308503e+06


In [8]:
wb = xw.books('B -PNL-vfx2-'+str(todaynum)+'.xlsx')
sht = wb.sheets['equity']

In [9]:
# 更改第一天權益
wb = xw.books('B -PNL-vfx2-'+str(todaynum)+'.xlsx')
sht = wb.sheets['equity']
df = equity_change[equity_change['nday']==1]
for i in range(len(df.index)):
    rownum = df.iloc[i, 1]
    value = df.iloc[i, 3:]
    # print(rownum)
    # print(type(value))
    # print(value.values)
    sht.range('D'+str(rownum)+":L"+str(rownum)).value = value.values
    sht.range('D'+str(rownum)+":L"+str(rownum)).color = (255,255,0)
# 更改第二天權益
sht = wb.sheets['equity_Y']
df = equity_change[equity_change['nday']==2]
for i in range(len(df.index)):
    rownum = df.iloc[i, 1]
    value = df.iloc[i, 3:]
    # print(rownum)
    # print(type(value))
    # print(value.values)
    sht.range('D'+str(rownum)+":L"+str(rownum)).value = value.values
    sht.range('D'+str(rownum)+":L"+str(rownum)).color = (255,255,0)

In [10]:
# 檢查是否有SWAP.csv 如果有則進行swap修正
files = [f for f in os.listdir('.') if os.path.isfile(f)]
r = re.compile("vfx2_swap.csv")
swap_list = list(filter(r.match, files)) # Read Note
swap_list
if len(swap_list)>0:
    # 2.檢查解歸檔 修正SWAP
    swap = pd.read_csv('vfx2_swap.csv')
    swap.head()
    # 如當日有解歸檔則進行swap修正
    if swap.shape[0] > 0:
        # 開啟swap sheet 
        sht = wb.sheets['swap']
        for i in range(swap.shape[0]):
            swap.loc[i,'excel_row']
            sht.range('I'+str(swap.loc[i,'excel_row'])).value = sht.range('J'+str(swap.loc[i,'excel_row'])).value
        print('解歸檔資料整理完畢')
else: 
    print('今日無解歸檔資料')

今日無解歸檔資料


In [11]:
# 讀取RT_value表
vfxRaw = pd.read_excel(str(todaynum)+'_RTvalue.xlsx' , sheet_name= 'VFX2_Raw')
vfxRaw.head()

,db,Login,Group,group,CCY,Deal,Type,Profit,Comment,IO,db2,Login2,Group2,group2,Date,VIG,MNT
0,VAU3,6202487,M_VIGM_10_USD,M,USD,2895169,balance,15.65,Transfer In from 66302,in,VAU3,66302,R_REBAT_VIG_USD,R,2021-01-25,1,1
1,VAU3,6204018,M_VIGT_P_USD,M,USD,2919848,balance,10.34,Transfer In from 6576,in,VAU2,6576,R_REBAT_VIG_USD,R,2021-01-25,1,1
2,VAU3,6204018,M_VIGT_P_USD,M,USD,2919853,balance,68.44,Transfer In from 63258,in,VAU2,63258,R_REBAT_VIG_USD,R,2021-01-25,1,1
3,VAU3,6204018,M_VIGT_P_USD,M,USD,2919856,balance,112.03,Transfer In from 66317,in,VAU3,66317,R_REBAT_VIG_USD,R,2021-01-25,1,1
4,VAU3,6204018,M_VIGT_P_USD,M,USD,2919866,balance,21.72,Transfer In from 68368,in,VAU3,68368,R_REBATBK_V_USD,R,2021-01-25,1,1


In [12]:
# 1.刪除GROUP含有CAN字串
# vfxRaw[vfxRaw['Group'].str.contains(r'CAN')]
# type(np.where(vfxRaw['Group'].str.contains(r'CAN'))[0])
can_rowsnumber = np.where(vfxRaw['Group'].str.contains(r'CAN'))[0]
# 2.刪除COMMENT含有/div
div_rowsnumber = np.where(vfxRaw['Comment'].str.contains(r'/div'))[0]
# 3.填補rollback group
vfxRaw['db2'].fillna(value=vfxRaw.groupby('Login')['db2'].unique(), inplace=True)
rollback = vfxRaw[vfxRaw['Comment'].str.contains(r'/Rollback')]
print(can_rowsnumber, div_rowsnumber)


[] []


In [13]:
# comment : Rollback
# vfxRaw['db2'].fillna(value=20, inplace=False)
vfxRaw[vfxRaw['Comment'].str.contains(r'/Rollback')]

,db,Login,Group,group,CCY,Deal,Type,Profit,Comment,IO,db2,Login2,Group2,group2,Date,VIG,MNT


In [15]:
# 3. RT_value 確認完畢後複製至vfx2_VIG sheet
# 修改RT_value表
wbcopy = xw.books(todaynum+'_RTvalue.xlsx')
# 使用工作表
vfx2_Result = wbcopy.sheets['Result']
my_values = vfx2_Result.range('G20:O22').options(ndim=2).value 
wb.sheets['VFX_VIG'].range('AQ10:AY12').value = my_values
print('FN Table修改完畢')

FN Table修改完畢


In [28]:
# 4. 複製貼上Rebate與手動修正
# 讀取sales_commission & rebate 合併計算
commission = pd.read_excel('VFX_AU3_UK3_sales_commission'+str(today)+'.xlsx' , sheet_name='usd')
rebate  = pd.read_excel('VFX_AU3_UK3_rebate_report'+str(today)+'.xlsx' , sheet_name='usd')
df = pd.concat([commission, rebate])
df = df.groupby(['mt4Account']).agg({'inner_commission': "sum", 'outer_commission': "sum"})
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'VFX_AU3_UK3_sales_commission2021-01-22.xlsx'

In [4]:
# # copy pd.df to excel
wb = xw.books('B -PNL-vfx2-'+todaynum+'.xlsx')
sht = wb.sheets['rebate']
rownumber_added = sht.range('A' + str(wb.sheets[0].cells.last_cell.row)).end('up').row + 1
sht.range("A"+str(rownumber_added)).options(index=True,header=False).value=df 

In [5]:
# vfx2
files = [f for f in os.listdir('.') if os.path.isfile(f)]
r = re.compile("^AU3 ")
newlist1 = list(filter(r.match, files)) # Read Note
r = re.compile("^UK3 ")
newlist2 = list(filter(r.match, files)) # Read Note
rebate_files = newlist1+newlist2
print(rebate_files)

[]


In [6]:
# 讀取需要手動更改之rebate
data = []
for i in rebate_files:
    print(i)
    login = int(i.split()[1].replace("-",""))
    wb = xw.Book(i)
    sht = wb.sheets[todaymd]
    # 刪除 rebate 含有NA的交易紀錄
    arrt = sht.range('AD:AD')[3:].value
    type(arrt)
    arrt =[x for x in arrt if x is not None]
    print('P'+str(len(arrt)+4)+':AG500')
    sht.range('P'+str(len(arrt)+4)+':AG500').clear()
    rebate_arr = [login, sht.range('M10').value, sht.range('N10').value]
    print(rebate_arr)
    data.append(rebate_arr)
    wb.close()
print(data)

[]


In [7]:
df = pd.DataFrame(data, columns=['mt4Account', 'inner_new', 'outer_new'])
df.head()

,mt4Account,inner_new,outer_new


In [13]:
wb = xw.books('B -PNL-vfx2-'+todaynum+'.xlsx')
sht = wb.sheets['rebate']
a = sht.range('A:A').value
for i in range(len(df.index)):
    if df['mt4Account'][i] in a:
        index = a.index(df['mt4Account'][i])
        # df 取代舊值
        print(sht.range('A'+str(index+1)).value)
        sht.range('B'+str(index+1)).value = df['inner_new'][i] + df['outer_new'][i]
        # sht.range('C'+str(index+1)).value = df['outer_new'][i]
        sht.range('A'+str(index+1)+":"+'C'+str(index+1)).color = (255,255,0)
    else:
        print(str(df['mt4Account'][i]) + ' is not in list')


91012.0
